# Logistic Regression Project - Predict Customer Churn

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv', inferSchema=True, header=True)

In [0]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [0]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [0]:
data.columns

Out[7]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

# Process Data

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(
    inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], 
    outputCol='features'
)

In [0]:
output = assembler.transform(data)

In [0]:
final_data = output.select('features', 'churn')

In [0]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

# Make Model

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
lr_churn = LogisticRegression(labelCol='churn')

In [0]:
fitted_churn_model = lr_churn.fit(train_churn)

In [0]:
training_sum = fitted_churn_model.summary

In [0]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               628|                628|
|   mean|0.1624203821656051|0.12898089171974522|
| stddev|0.3691300195355856| 0.3354459712595925|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



# Evaluate Model

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [0]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.75325076519386...|[0.99145011469248...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.69619877473993...|[0.66734445345898...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.60618167531152...|[0.83288058716116...|       0.0|
|[28.0,11128.95,1....|    0|[4.31026753748726...|[0.98674801744702...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.80561775282426...|[0.94297864301332...|       0.0|
|[29.0,10203.18,1....|    0|[3.89746911026662...|[0.98011041703660...|       0.0|
|[29.0,13240.01,1....|    0|[6.90694962754584...|[0.99900019463512...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.28870222946973...|[0.98646304123965...|       0.0|
|[30.0,10960.52,1....|    0|[2.50816210867143...|[0.92471203693650...|       0.0|
|[30.0,12788.37,

In [0]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [0]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [0]:
auc

Out[23]: 0.7008928571428572

# Predict New Data

In [0]:
final_lr_model = lr_churn.fit(final_data)

In [0]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv', inferSchema=True, header=True)

In [0]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [0]:
test_new_customers = assembler.transform(new_customers)

In [0]:
final_results = final_lr_model.transform(test_new_customers)

In [0]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [0]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          